# Environment Setup #

In [1]:
!pip uninstall protobuf -y
!pip install protobuf==3.20.3
!pip install tensorflow==2.17.1 tensorflow-metadata==1.13.1 protobuf==3.20.3 chromadb==0.5.23
!pip install chromadb sentence-transformers

!pip install gdown
!pip install PyPDF2
!pip install youtube-transcript-api
!pip install langchain
!pip install -U langchain-community

!apt-get remove -y chromium-browser chromium-chromedriver
!rm -f /usr/bin/chromedriver

!pip install selenium
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver
!ln -s /usr/lib/chromium-browser/chromedriver /usr/bin/chromedriver

!pip install wikidata
!pip install langdetect
!pip install deep_translator

!pip install llama-index --upgrade
!pip install sentence-transformers --upgrade

!pip install python-decouple==3.8 llm-templates

Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


  Using cached youtube_transcript_api-0.6.3-py3-none-any.whl.metadata (17 kB)
Using cached youtube_transcript_api-0.6.3-py3-none-any.whl (622 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'chromium-browser' is not installed, so not removed
Package 'chromium-chromedriver' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 25.0 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cu

In [65]:
import gdown
import os
import shutil
import re
import PyPDF2
import chromadb
import pandas as pd
import time
import requests

# YouTube API
from youtube_transcript_api import YouTubeTranscriptApi

# Langchain to manage chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Transformers
from sentence_transformers import SentenceTransformer

# Web Scrapping with Selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Sklearn for machine learning models
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# LLM with api key
from huggingface_hub import InferenceClient
from google.colab import userdata

# Language Manager
from langdetect import detect
from deep_translator import GoogleTranslator

# Excersie 1 #

In this code I will generate a chatbot trained with some content, ultimately expert in Rajas of the Ganges eurogame (board game).

# Step 1: Content Download #

## Text context: rules and review ##

### Download PDF ###

In [66]:
drive_folder_url = "https://drive.google.com/drive/folders/1bMK7C8qbU96FvUjOBT1D35wy1LZw3K4E?usp=sharing"

# Descarga carpeta 'game_data'
gdown.download_folder(drive_folder_url, quiet=True, output='game_data')

['game_data/Rajas_of_the_Ganges_-_rules_in_brief_v1.pdf',
 'game_data/Rajas_of_the_Ganges_Plain_and_Simple.pdf',
 'game_data/Rajas_of_the_Ganges_Quick_Rules_Guide.pdf',
 'game_data/rules_EN.pdf']

Extraer texto

In [67]:
def remove_uppercase(x):
    x1 = x.lower()
    return x1

def remove_signs(x):
    x1 = re.sub(r'[^\w\s]', '', x)  # Elimina los signos, dejando solo palabras y espacios
    return x1.strip()

def remove_spaces(x):
    x1 = re.sub(r'\s+', ' ', x).strip()  # Reemplaza múltiples espacios con uno solo
    return x1

def clean_text(text):
    # Eliminar tabulaciones y dividir en oraciones
    text = text.replace("\t", "") #elimino tabs
    text = text.replace("\n", " ") #cambio saltos de línea por espacios

    # Aplicar las transformaciones en el orden correcto
    output = remove_uppercase(remove_signs(remove_spaces(text)))

    return output

In [68]:
# Abre el archivo PDF
pdf_path = 'game_data/Rajas_of_the_Ganges_-_rules_in_brief_v1.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    context1 = ''

    # Extrae el texto de cada página
    for page in reader.pages:
        context1 += page.extract_text()

In [69]:
doc1 = clean_text(context1)
print(f"Longitud de doc1: {len(doc1)}")

Longitud de doc1: 3542


In [70]:
# Abre el archivo PDF
pdf_path = 'game_data/Rajas_of_the_Ganges_Plain_and_Simple.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    context2 = ''

    # Extrae el texto de cada página
    for page in reader.pages:
        context2 += page.extract_text()

In [71]:
doc2 = clean_text(context2)
print(f"Longitud de doc2: {len(doc2)}")

Longitud de doc2: 13169


In [72]:
# Abre el archivo PDF
pdf_path = 'game_data/Rajas_of_the_Ganges_Quick_Rules_Guide.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    context3 = ''

    # Extrae el texto de cada página
    for page in reader.pages:
        context3 += page.extract_text()

In [73]:
doc3 = clean_text(context3)
print(f"Longitud de doc3: {len(doc3)}")

Longitud de doc3: 13815


In [74]:
# Abre el archivo PDF
pdf_path = 'game_data/rules_EN.pdf'
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    context4 = ''

    # Extrae el texto de cada página
    for page in reader.pages:
        context4 += page.extract_text()

In [75]:
doc4 = clean_text(context4)
print(f"Longitud de doc4: {len(doc4)}")

Longitud de doc4: 35431


### Download YouTube video ###

In [76]:
video_id_1 = "OJ0anCDdIJ4"
video_id_2 = "ruScGyu1_5w"

# Obtener el transcript
try:
    transcript1 = YouTubeTranscriptApi.get_transcript(video_id_1)
    transcript2 = YouTubeTranscriptApi.get_transcript(video_id_2)

    # Convertir el transcript en una sola cadena de texto
    transcript_text_1 = " ".join([entry["text"] for entry in transcript1])
    transcript_text_2 = " ".join([entry["text"] for entry in transcript2])

    context5 = transcript_text_1
    context6 = transcript_text_2
except Exception as e:
    print(f"Error al obtener el transcript: {e}")

In [77]:
doc5 = clean_text(context5)
print(f"Longitud de doc5: {len(doc5)}")

doc6 = clean_text(context6)
print(f"Longitud de doc6: {len(doc6)}")

Longitud de doc5: 11173
Longitud de doc6: 65091


### Chunking ###

In [78]:
# Concatenar todo el contexto en un solo texto
docs = [doc1, doc2, doc3, doc4, doc5, doc6]
texts, metadatas, ids = [], [], []

for i in range(len(docs)):
  # Dividir el texto en chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
  chunks = text_splitter.split_text(docs[i])

  for chunk in chunks:
    texts.append(chunk)
    if i<5:
      metadatas.append({"document":f"doc{i+1}", "type":"rules"})
    else:
      metadatas.append({"document":f"doc{i+1}", "type":"review"})
    ids.append(f"doc{i+1}_{chunks.index(chunk)}")

print(len(texts))
print(len(metadatas))
print(len(ids))

for i in range(10):
  print(texts[i])

318
318
318
rajas of the ganges 2017 24 players designer inka brand  marcus brand duration  players money and fame markers cross 45 75 minutes board setup  pick the game board side for  of players  2p or 3p4p 3p  use cover tiles to cover marked spaces  sort province tiles by color then by symbol snake cow tiger to get 12 shuffled stacks  shuffle 8 white yield tiles and set face down stack on the temple player setup  each player gets a province board  kali statue board  6 workers  5 cubes  1 money marker  1
board  6 workers  5 cubes  1 money marker  1 fame marker  and 1 bonus marker for the money track  place 3 total workers on the fame track 15 space money track 20 space and river on bridge  place boat on starting space of the river put karma cube of level 1 of karma path put 1 upgrade cube in left column value 2 of each building development scroll  put 1 fame marker on space 0 of fame track put bonus marker with the river space side up on money tra ck space 12  each player takes and r

### Chroma DB ###

In [79]:
# BBDD
client = chromadb.Client()
try:
  client.delete_collection('vector_db')
except Exception:
  pass
vector_db = client.create_collection("vector_db")

# Embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Iteration over chunks
# texts metadatas ids
for i in range(len(texts)):
  embedding = embedding_model.encode(texts[i])
  vector_db.add(documents=[texts[i]], metadatas=[metadatas[i]], ids=[ids[i]], embeddings=[embedding])
  print(f"Id: {ids[i]} - Embedding length: {len(embedding)} - Metadata: {metadatas[i]}")


# TESTING
results = vector_db.query(query_texts=["players"], n_results=1)
print(results)

Id: doc1_0 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_1 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_2 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_3 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_4 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_5 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_6 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc1_7 - Embedding length: 384 - Metadata: {'document': 'doc1', 'type': 'rules'}
Id: doc2_0 - Embedding length: 384 - Metadata: {'document': 'doc2', 'type': 'rules'}
Id: doc2_1 - Embedding length: 384 - Metadata: {'document': 'doc2', 'type': 'rules'}
Id: doc2_2 - Embedding length: 384 - Metadata: {'document': 'doc2', 'type': 'rules'}
Id: doc2_3 - Embedding length: 384 - Metadata: {'document': 'doc2

In [80]:
results = vector_db.query(query_texts=["amount of players"], n_results=1)
print(results)

{'ids': [['doc6_135']], 'embeddings': None, 'documents': [['on the opposite side is for three and four just because of some of the spaces theres a little bit more space right and so in terms of player count how do you feel uh plays well at all player counts yeah we played at all players we have played at all player accounts uh where is my favorite maybe three i want to say maybe uh it plays very well at two though its nice to be able to go you know back to back i will say that when higher player counts theres the same amount of tiles out there and so']], 'uris': None, 'data': None, 'metadatas': [[{'document': 'doc6', 'type': 'review'}]], 'distances': [[0.9863834977149963]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [81]:
print(results["documents"][0][0])

on the opposite side is for three and four just because of some of the spaces theres a little bit more space right and so in terms of player count how do you feel uh plays well at all player counts yeah we played at all players we have played at all player accounts uh where is my favorite maybe three i want to say maybe uh it plays very well at two though its nice to be able to go you know back to back i will say that when higher player counts theres the same amount of tiles out there and so


## Dataframe context: stats##

In [82]:
def game_stats():
  # Configuración de las opciones de Selenium para Chrome
  chrome_options = Options()
  chrome_options.add_argument("--headless")  # Ejecutar sin interfaz gráfica
  chrome_options.add_argument("--no-sandbox")  # Para entornos sin GUI
  chrome_options.add_argument("--disable-dev-shm-usage")  # Para evitar problemas de memoria
  chrome_options.binary_location = '/usr/bin/chromium-browser'  # Ruta al binario de Chromium

  # Crear el driver de Selenium
  driver = webdriver.Chrome(options=chrome_options)

  # URL de la página a scrapeo
  url = "https://boardgamegeek.com/boardgame/220877/rajas-of-the-ganges/stats"
  driver.get(url)

  # Esperar a que se cargue el contenido dinámico
  time.sleep(3)

  # Encontrar los elementos en la página
  titles_elements = driver.find_elements(By.CLASS_NAME, 'outline-item-title')
  descriptions_elements = driver.find_elements(By.CLASS_NAME, 'outline-item-description')

  # Extraer texto
  titles = [title.text.strip() for title in titles_elements]
  descriptions = [description.text.strip() for description in descriptions_elements]

  # Crear un DataFrame con los datos
  data = {'Title': titles, 'Description': descriptions}
  game_stats = pd.DataFrame(data)

  formatted_string = '\n'.join(game_stats.apply(lambda row: f"{row['Title']}: {row['Description']}", axis=1))

  # Cerrar el navegador
  driver.quit()
  return(formatted_string)

print(game_stats())

Avg. Rating: 7.731
No. of Ratings: 14,702
Std. Deviation: 1.13
Weight: 2.89 / 5
Comments: 2,125
Fans: 691
Page Views: 899,914
Overall Rank: 156
Historical Rank
Strategy Rank: 123
Historical Rank
All Time Plays: 59,439
This Month: 232
Own: 19,560
Prev. Owned: 2,058
For Trade: 194
Find For-Trade Matches
Want In Trade: 676
Find Want-in-Trade Matches
Wishlist: 3,810
Has Parts: 11
Want Parts: 4


## Knowledge Graph context: related games ##

In [83]:
def get_wikidata_info(entity_id):
    sparql_endpoint = "https://query.wikidata.org/sparql"

    # Consulta SPARQL ajustada
    query = f"""
    SELECT ?property ?propertyLabel ?value ?valueLabel
    WHERE {{
        wd:{entity_id} ?property ?value.
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(sparql_endpoint, params={'query': query, 'format': 'json'}, headers=headers)
    data = response.json()

    # Diccionario para almacenar la información relevante
    properties_dict = {
    "http://www.wikidata.org/prop/direct/P31":"instance of",
    "http://www.wikidata.org/prop/direct/P123":"publisher",
    "http://www.wikidata.org/prop/direct/P178":"developer",
    "http://www.wikidata.org/prop/direct/P577":"publication date",
    }

    # Diccionario donde almacenaremos la información filtrada
    game_wikidata = {}

    # Recorrer los resultados
    for result in data['results']['bindings']:
        property_uri = result['property']['value']

        # Verificar si la propiedad está en properties_dict
        if property_uri in properties_dict:
            property_label = properties_dict[property_uri]
            # Si existe el valor en 'valueLabel', lo usamos; de lo contrario, usamos el valor crudo
            value_label = result.get('valueLabel', {}).get('value', result['value']['value'])

            # Guardamos la información en el diccionario game_wikidata
            game_wikidata[property_label] = value_label

    # Imprimir el diccionario resultante
    print(game_wikidata)
    return game_wikidata

# Consulta el artículo con ID Q56759854
game_wikidata = get_wikidata_info("Q56759854")


{'instance of': 'board game', 'publisher': 'HUCH!', 'developer': 'Inka and Markus Brand', 'publication date': '2017-01-01T00:00:00Z'}


### Games with same publisher ###

In [84]:
def games_same_publisher():
    sparql_endpoint = "https://query.wikidata.org/sparql"

    # Consulta SPARQL usando el URI completo de P123 (editorial) y Q1566108 (HUCH!)
    query = """
    SELECT ?item ?itemLabel ?publisherLabel
    WHERE {
      ?item <http://www.wikidata.org/prop/direct/P123> <http://www.wikidata.org/entity/Q1566108>.  # HUCH! como editorial
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(sparql_endpoint, params={'query': query, 'format': 'json'}, headers=headers)
    data = response.json()

    # Imprimir los resultados
    same_publisher = "Games of the same publisher:\n"
    if data['results']['bindings']:
      for result in data['results']['bindings']:
          item_label = result['itemLabel']['value']
          publisher_label = result.get('publisherLabel', {}).get('value', "Unknown Publisher")
          item_id = result['item']['value']
          same_publisher = same_publisher+item_label+"\n"
      return same_publisher
    else:
      print("No results found for HUCH! publisher.")
      return same_publisher+"Not found."

# Ejecutar la consulta
same_publisher = games_same_publisher()
print(same_publisher)


Games of the same publisher:
YINSH
Ta-Ke
Cookies
Djinn
Rajas of the Ganges
Q60983282
Nut Job
Espresso Doppio



### Games with same developer ###

In [85]:
def games_same_developer():
    sparql_endpoint = "https://query.wikidata.org/sparql"

    # Consulta SPARQL usando el URI completo de P178 (desarrollador) y Q1663646 (Inka and Markus Brand)
    query = """
    SELECT ?item ?itemLabel ?developerLabel
    WHERE {
      ?item <http://www.wikidata.org/prop/direct/P178> <http://www.wikidata.org/entity/Q1663646>.  # Inka and Markus Brand como desarrollador
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(sparql_endpoint, params={'query': query, 'format': 'json'}, headers=headers)
    data = response.json()

    # Imprimir los resultados
    same_developer = "Games of the same developer:\n"
    if data['results']['bindings']:
      for result in data['results']['bindings']:
          item_label = result['itemLabel']['value']
          developer_label = result.get('developerLabel', {}).get('value', "Unknown Developer")
          item_id = result['item']['value']
          same_developer = same_developer + item_label + "\n"
      return same_developer
    else:
      print("No results found for Inka and Markus Brand as developer.")
      return same_developer+"Not found."

# Ejecutar la consulta
same_developer = games_same_developer()
print(same_developer)


Games of the same developer:
Village
Burg der 1000 Spiegel
Monster Trap
Rajas of the Ganges



# Step 2: Classifier #

## Random Forest Model##

In [86]:
# Define categories and prompts explicitly
mock_data = [
    # Rules category
    ("What are the rules for placing workers in Rajas of the Ganges?", "rules"),
    ("How does the river mechanic work in Rajas of the Ganges?", "rules"),
    ("Can you explain the scoring system in Rajas of the Ganges?", "rules"),
    ("What happens if two players tie in Rajas of the Ganges?", "rules"),
    ("How do I gain karma in Rajas of the Ganges?", "rules"),
    ("What are the rules for using dice in Rajas of the Ganges?", "rules"),
    ("How does a player win in Rajas of the Ganges?", "rules"),
    ("What is the setup process for Rajas of the Ganges?", "rules"),
    ("How do I use the bonus tiles in Rajas of the Ganges?", "rules"),
    ("Can I gain points through the trade mechanism in Rajas of the Ganges?", "rules"),
    ("How does the taxation mechanic work in Rajas of the Ganges?", "rules"),
    ("What do the different colors of dice mean in Rajas of the Ganges?", "rules"),
    ("Can you explain how the river and palace tracks are connected?", "rules"),
    ("How do I get to use the special actions in Rajas of the Ganges?", "rules"),
    ("What are the different types of buildings in Rajas of the Ganges?", "rules"),
    ("How does the market mechanic work in Rajas of the Ganges?", "rules"),
    ("What is the role of the player board in Rajas of the Ganges?", "rules"),
    ("How do I earn money in Rajas of the Ganges?", "rules"),
    ("What do the resources in Rajas of the Ganges represent?", "rules"),
    ("How does the prestige track work in Rajas of the Ganges?", "rules"),
    ("What is the importance of the river in Rajas of the Ganges?", "rules"),
    ("How do you activate special abilities in Rajas of the Ganges?", "rules"),
    ("How do I gain points from the karma track in Rajas of the Ganges?", "rules"),
    ("What happens if I go over the maximum number of workers?", "rules"),
    ("How do I use the dice placement in Rajas of the Ganges?", "rules"),
    ("How does the taxation mechanic work in Rajas of the Ganges?", "rules"),
    ("Can I choose which dice to place in Rajas of the Ganges?", "rules"),
    ("How do I gain victory points in Rajas of the Ganges?", "rules"),
    ("What is the importance of the palace in Rajas of the Ganges?", "rules"),
    ("How do I use the river and palace tracks in Rajas of the Ganges?", "rules"),

    # Review category
    ("What do players think of Rajas of the Ganges?", "review"),
    ("Is Rajas of the Ganges a good game for beginners?", "review"),
    ("What are the pros and cons of Rajas of the Ganges?", "review"),
    ("Why do people like Rajas of the Ganges?", "review"),
    ("How does Rajas of the Ganges compare to other worker placement games?", "review"),
    ("Is Rajas of the Ganges worth buying?", "review"),
    ("What makes Rajas of the Ganges stand out among worker placement games?", "review"),
    ("Is Rajas of the Ganges considered a thematic game?", "review"),
    ("What is the player experience like in Rajas of the Ganges?", "review"),
    ("How does Rajas of the Ganges handle player interaction?", "review"),
    ("Is Rajas of the Ganges a good game for strategy enthusiasts?", "review"),
    ("What makes Rajas of the Ganges a fun game to play repeatedly?", "review"),
    ("How is the replayability of Rajas of the Ganges?", "review"),
    ("Is Rajas of the Ganges more luck-based or strategy-based?", "review"),
    ("What kind of player would enjoy Rajas of the Ganges?", "review"),
    ("How would you rate the components of Rajas of the Ganges?", "review"),
    ("What makes the river track mechanic interesting in Rajas of the Ganges?", "review"),
    ("How does the game balance strategy and randomness?", "review"),
    ("Is Rajas of the Ganges suitable for solo play?", "review"),
    ("How long does a typical game of Rajas of the Ganges last?", "review"),
    ("Is Rajas of the Ganges a good game for small groups?", "review"),
    ("How does the game compare to other games by the same designer?", "review"),
    ("What makes the karma track interesting in Rajas of the Ganges?", "review"),
    ("How does the dice mechanic influence gameplay in Rajas of the Ganges?", "review"),
    ("What is the difficulty level of Rajas of the Ganges?", "review"),
    ("How does Rajas of the Ganges compare to other medium-weight games?", "review"),
    ("Is Rajas of the Ganges a game that rewards long-term planning?", "review"),
    ("How accessible are the rules of Rajas of the Ganges?", "review"),

    # Stats category
    ("What is the average playtime of Rajas of the Ganges?", "stats"),
    ("How many players can play Rajas of the Ganges?", "stats"),
    ("What is the ideal player count for Rajas of the Ganges?", "stats"),
    ("What is the current ranking of Rajas of the Ganges on BoardGameGeek?", "stats"),
    ("How difficult is Rajas of the Ganges to learn?", "stats"),
    ("How long does it take to set up Rajas of the Ganges?", "stats"),
    ("What is the average duration of a game of Rajas of the Ganges?", "stats"),
    ("What is the weight of Rajas of the Ganges on BoardGameGeek?", "stats"),
    ("What is the player count range for Rajas of the Ganges?", "stats"),
    ("How long do games typically last with two players?", "stats"),
    ("What is the best player count for Rajas of the Ganges?", "stats"),
    ("How does player count affect the gameplay experience?", "stats"),
    ("What is the setup time for Rajas of the Ganges?", "stats"),
    ("What is the age recommendation for Rajas of the Ganges?", "stats"),
    ("What is the minimum player count for Rajas of the Ganges?", "stats"),
    ("How much downtime is there between turns in Rajas of the Ganges?", "stats"),
    ("What is the skill level required to play Rajas of the Ganges?", "stats"),
    ("What is the time required to explain the rules of Rajas of the Ganges?", "stats"),
    ("How much time does it take to complete a solo game of Rajas of the Ganges?", "stats"),
    ("How does the game scale with different player counts?", "stats"),
    ("What is the maximum number of players for Rajas of the Ganges?", "stats"),
    ("What is the minimum age requirement for Rajas of the Ganges?", "stats"),
    ("What is the total number of components in Rajas of the Ganges?", "stats"),
    ("How many different types of workers are there in Rajas of the Ganges?", "stats"),

    # Related Games category
    ("What games are similar to Rajas of the Ganges?", "related_games"),
    ("Can you recommend games like Rajas of the Ganges?", "related_games"),
    ("Are there any other games by the same designers as Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to Agricola?", "related_games"),
    ("Is Rajas of the Ganges similar to Stone Age?", "related_games"),
    ("What games are good follow-ups to Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to Le Havre?", "related_games"),
    ("What is the difference between Rajas of the Ganges and Tigris & Euphrates?", "related_games"),
    ("How does Rajas of the Ganges differ from Puerto Rico?", "related_games"),
    ("Is Rajas of the Ganges similar to Concordia?", "related_games"),
    ("Can you compare Rajas of the Ganges to Imperial Settlers?", "related_games"),
    ("Is Rajas of the Ganges a good alternative to Caverna?", "related_games"),
    ("How does Rajas of the Ganges compare to the game Clans of Caledonia?", "related_games"),
    ("What games have similar mechanics to Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to The Estates?", "related_games"),
    ("What are other worker placement games similar to Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to other strategy games?", "related_games"),
    ("Can you suggest other games that combine resource management and worker placement?", "related_games"),
    ("How does Rajas of the Ganges compare to Castles of Burgundy?", "related_games"),
    ("What other games feature a river mechanic like Rajas of the Ganges?", "related_games"),
    ("What games have similar dice mechanics to Rajas of the Ganges?", "related_games"),
    ("Are there other games with karma as a mechanic like Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to Great Western Trail?", "related_games"),
    ("What are some games that focus on economy and expansion like Rajas of the Ganges?", "related_games"),
    ("How does Rajas of the Ganges compare to Gaia Project?", "related_games")
]

# Create the DataFrame
training_data = pd.DataFrame(mock_data, columns=["prompt", "type"])

# Display the DataFrame
training_data

,prompt,type
0,What are the rules for placing workers in Raja...,rules
1,How does the river mechanic work in Rajas of t...,rules
2,Can you explain the scoring system in Rajas of...,rules
3,What happens if two players tie in Rajas of th...,rules
4,How do I gain karma in Rajas of the Ganges?,rules
...,...,...
102,What games have similar dice mechanics to Raja...,related_games
103,Are there other games with karma as a mechanic...,related_games
104,How does Rajas of the Ganges compare to Great ...,related_games
105,What are some games that focus on economy and ...,related_games


In [87]:
# Convertir las preguntas (prompts) en vectores de características usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(training_data['prompt'])

# Las etiquetas son las categorías
y = training_data['type']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Evaluar el modelo
y_pred = random_forest.predict(X_test)

# Mostrar las estadísticas de evaluación
print(classification_report(y_test, y_pred))


               precision    recall  f1-score   support

related_games       0.75      1.00      0.86         3
       review       0.50      0.33      0.40         6
        rules       0.55      1.00      0.71         6
        stats       0.33      0.14      0.20         7

     accuracy                           0.55        22
    macro avg       0.53      0.62      0.54        22
 weighted avg       0.49      0.55      0.48        22



In [88]:
# TESTING
prompt = ["What is the importance of karma in Rajas of the Ganges?"]
embedded = vectorizer.transform(prompt)
prediction = random_forest.predict(embedded)
print("Prediction:", prediction[0])

prompt = ["How many people plays the game?"]
embedded = vectorizer.transform(prompt)
prediction = random_forest.predict(embedded)
print("Prediction:", prediction[0])


Prediction: rules
Prediction: review


## LLM Model ##

In [102]:
types=["rules", "review", "stats", "related_games"]

client = InferenceClient(api_key=userdata.get('HF_TOKEN'))
role="user"
content=f"""You are an assistant for a board game called Rajas of the Ganges.
Your task is to classify each prompt (usually a question) into exactly one of the following four categories:
rules, review, stats, related_games.
Respond with one and only one of these four categories, using only the category name as your response.
Do not include any additional text, characters, or formatting. Your answer is either 'rules', 'review', 'stats' or 'related_games'.
This is the question you should clasify:
"""

def llm_classifier(prompt):
  message=[{"role": role, "content": content+prompt}]
  completion = client.chat.completions.create(model="Qwen/Qwen2.5-72B-Instruct", messages=message, max_tokens=200)
  classification = completion.choices[0].message.content.strip()
  if str(classification) not in types:
    print(f"Random classification, llm clasified {classification}.")
    classification = types[len(prompt)%3]
  print(classification.lower())
  return classification.lower()

print(llm_classifier("What other games feature a river mechanic like Rajas of the Ganges?"))

related_games
related_games


In [103]:
sum=0
for i in range(training_data.shape[0]):
  llm_prediction = llm_classifier(training_data["prompt"][i])
  time.sleep(1)
  if llm_prediction==training_data["type"][i]:
    sum+=1

print(sum/training_data.shape[0])

rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
rules
review
review
review
review
related_games
review
related_games
Random classification.
stats
review
rules
review
review
review
stats
related_games
review
rules
rules
review
rules
review
related_games
rules
rules
stats
related_games
rules
rules
stats
stats
related_games
review
review
rules
stats
stats
rules
stats
stats
rules
rules
stats
stats
rules
stats
rules
rules
rules
stats
rules
stats
rules
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
Random classification.
rules
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
related_games
0.719626168224299


# Step 3: Retriever #

## Retrievers y Reranker ##

In [104]:
from sentence_transformers import CrossEncoder

class Reranker:
    def __init__(self, model_name: str = 'cross-encoder/ms-marco-MiniLM-L-6-v2'):
        # Initializes the reranker with a cross-encoder model.
        self.model = CrossEncoder(model_name)

    def rerank(self, query: str, nodes, top_k: int = None) -> list:
        # Re-ranks the retrieved nodes using the cross-encoder.
        if not nodes:
            return nodes

        pairs = [(query, node) for node in nodes]
        scores = self.model.predict(pairs)
        scored_nodes = list(zip(scores, nodes))
        scored_nodes.sort(key=lambda x: x[0], reverse=True)

        if top_k:
            scored_nodes = scored_nodes[:top_k]

        return [node for _, node in scored_nodes]


def retriever_vector_db(prompt):
    reranker = Reranker()

    # Retrieves the most relevant document from the vector database.
    results = vector_db.query(query_texts=[prompt], n_results=10)  # Retrieve more results for reranking
    nodes = [doc[0] for doc in results['documents']]  # Extract documents

    if reranker:
        # Apply reranking if a reranker is provided.
        nodes = reranker.rerank(prompt, nodes)

    return nodes[0] if nodes else None

def retriever_dataframe_db():
  context = game_stats()
  return context

def retriever_graph_db():
  same_developer = games_same_developer()
  same_publisher = games_same_publisher()
  context = same_developer+"\n"+same_publisher
  return context

# This function calls the ones already defined, and provide the appropriate context for an specific prompt.
def retriever(prompt):
  classification = llm_classifier(prompt)

  if classification=="rules":
    context = retriever_vector_db(prompt)
    return context
  elif classification=="review":
    context = retriever_vector_db(prompt)
    return context
  elif classification=="stats":
    context = retriever_dataframe_db()
    return context
  elif classification=="related_games":
    context = retriever_graph_db()
    return context
  else:
    print("Clasification failed. Sending vector db context.")
    context = retriever_vector_db(prompt)
    return context

In [105]:
# TEST
# Initialize the reranker
print('Initializing reranker...')


# Example usage
query = "How should I start the game board?"
context = retriever(query)
print("Retrieved context:", context)

Initializing reranker...
rules
Retrieved context: have retrieved one or both of their extra works from the board at the end of the round the start player marker transfers to either the person who played on the great mogul or to the person to the left of the current start player all players retrieve their active workers and game play resumes with the new start player on their turn the player will take one of their meeples and place it on the board immediately paying the cost and performing the action of that space both the cost and the benefit


# Step 4: Prompt Preparation #

## Language Manager ##

In [106]:
def isSpanish(input):
  idioma = detect(input)
  if idioma=="es":
    return True
  elif idioma=="en":
    return False
  else:
    return None

print(isSpanish("Hola, ¿cómo estás?"))
print(isSpanish("Hello, how are you?"))

translated = GoogleTranslator(source='en', target='es').translate('I want to translate this text')
print(translated)


def translate_to_english(content):
  if type(content)==str:
    translated = GoogleTranslator(source='es', target='en').translate(content)
    return translated
  else:
    return None

def translate_to_spanish(content):
  if type(content)==str:
    translated = GoogleTranslator(source='en', target='es').translate(content)
    return translated
  else:
    return None

print(translate_to_english("Hola, mi nombre es Francisco."))
print(translate_to_spanish("Hello, my name is Francisco."))

True
False
Quiero traducir este texto
Hello, my name is Francisco.
Hola, mi nombre es Francisco.


## Prompt and execution ##

In [112]:
def prompt_prep(context, user_input):
  query = f"""You're helping an end user with a board game called Rajas of the Ganges.
  You're a subject matter expert that will provide an answer to, ideally, a single question from this user.
  Your answer has to be as much direct and consistant as possible.
  If the context available is not enough to answer the question, ask the end user to reformulate the question.
  This is the context available:
  {context}
  This is the end user question:
  {user_input}
  """
  return query

def generate_answer(context, user_input):
    # Generates an answer to the given query based on the provided context using the specified model.
    client = InferenceClient(api_key=userdata.get('HF_TOKEN'))  # Initialize the inference client

    model = "Qwen/Qwen2.5-72B-Instruct"
    max_tokens=100

    try:
        # Build the prompt with the query and context
        prompt = prompt_prep(context, user_input)

        # Request a response from the model
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=0.7,
            top_p=0.95
        )

        # Extract and return the generated content
        return response.get('choices', [{}])[0].get('message', {}).get('content', "")

    except Exception as error:
        # Handle any errors that occur during the process
        return "An error occurred while processing your request."

In [115]:
def help():
  print("Rajas of the Ganges - HELP!")
  user_input = str(input("Ask me something: "))

  if isSpanish(user_input):
    print("Así que hablas español. Deja que me preparo para la ocasión. *se prepara un mate*")
    user_input_english = translate_to_english(user_input)

    print("Buscando en esta base de datos:")
    context = retriever(user_input_english)
    output = generate_answer(context, user_input_english)
    output_spanish = translate_to_spanish(output)

    print("Mi respuesta: ")
    print(output_spanish)
  else:
    print("Searching in this database:")
    context = retriever(user_input)

    print("Let me think...")
    output = generate_answer(context, user_input)

    print("This is my answer: ")
    print(output)

In [116]:
help()

Rajas of the Ganges - HELP!
Ask me something: How many players do I need to play the game?
Searching in this database:
rules
Let me think...
This is my answer: 
The game Rajas of the Ganges can be played with 2 to 4 players. It plays well at all player counts, but it's particularly enjoyable with 3 players, and it's also nice to play with 2 players because you can take turns back to back.


In [117]:
help()

Rajas of the Ganges - HELP!
Ask me something: ¿cuántos jugadores necesito para comenzar una partida?
Así que hablas español. Deja que me preparo para la ocasión. *se prepara un mate*
Buscando en esta base de datos:
rules
Mi respuesta: 
Para empezar una partida de Rajas of the Ganges, se necesitan 2 jugadores o entre 3 y 4 jugadores. El tablero de juego tiene configuraciones específicas para estas cantidades de jugadores, indicadas por los elefantes en la parte inferior derecha del tablero. Asegúrate de utilizar el lado apropiado del tablero en función de la cantidad de jugadores que tengas.


In [118]:
help()

Rajas of the Ganges - HELP!
Ask me something: What is the avr. raiting for this game?
Searching in this database:
review
Let me think...
This is my answer: 
I'm afraid the context provided doesn't include any information about the average rating of the game "Rajas of the Ganges." Could you please provide more details or reformulate your question to specify what kind of rating you're looking for (e.g., player experience, complexity, fun factor)? This will help me give you a more accurate answer.


In [119]:
help()

Rajas of the Ganges - HELP!
Ask me something: regarding statistics, what is the avr. raiting for this game?
Searching in this database:
stats
Let me think...
This is my answer: 
The average rating for "Rajas of the Ganges" is 7.731.


In [120]:
help()

Rajas of the Ganges - HELP!
Ask me something: Me encanto el juego. Recomiendame juegos similares.
Así que hablas español. Deja que me preparo para la ocasión. *se prepara un mate*
Buscando en esta base de datos:
related_games
Mi respuesta: 
¡Me alegra saber que disfrutaste de *Rajas of the Ganges*! Si estás buscando juegos similares, es posible que te gusten los siguientes basados ​​en los elementos estratégicos y temáticos de *Rajas of the Ganges*:

1. **Village**: este juego, del mismo desarrollador, ofrece una combinación de gestión de recursos y ubicación estratégica, similar a la profundidad estratégica que quizás hayas disfrutado en *Rajas of the Ganges*.

2. **Burg der 100


# Excercise 2 #

In [142]:
# Ollama
!curl -fsSL https://ollama.com/install.sh | sh

!rm -f ollama_start.sh
!echo '#!/bin/bash' > ollama_start.sh
!echo 'ollama serve' >> ollama_start.sh

# Make the script executable
!chmod +x ollama_start.sh
!nohup ./ollama_start.sh &
!ollama pull llama3.2 > ollama.log
!pip install llama-index-llms-ollama llama-index
!nohup litellm --model ollama/llama3.2:latest --port 8000 > litellm.log 2>&1 &

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 34bb5ab01051... 100% ▕▏  561 B                        

In [143]:
time.sleep(20)

In [144]:
# Ollama
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.formatter import ReActChatFormatter

In [149]:
def agent_help():
  print("Rajas of the Ganges - AGENT HELP!")
  user_input = str(input("Ask me something: "))

  # LLM Setup
  llm = Ollama(model="llama3.2:latest", request_timeout=100, temperature=0.1, context_window=4000)
  Settings.llm = llm

  # Creating tools
  tools = [FunctionTool.from_defaults(fn=retriever_vector_db, description="Information about rules and reviews.")]

  #FunctionTool.from_defaults(fn=retriever_dataframe_db, description="Information about game statistics."),
  #FunctionTool.from_defaults(fn=retriever_graph_db, description="Information about related games."),

  # Crear el agente ReAct
  agent = ReActAgent.from_tools(
      tools,
      llm=llm,
      verbose=True,
      chat_formatter=ReActChatFormatter(),
      system_prompt=
      """"
      You're helping an end user with a board game called Rajas of the Ganges.\n
      You're a subject matter expert that will provide an answer to, ideally, a single question from this user.\n
      Your answer has to be as much direct and consistant as possible.\n
      If the context available is not enough to answer the question, ask the end user to reformulate the question.\n

      You must follow this format:

      Thought: Here, I explain what I need to do.
      Action: nameOfTheTool
      Action Input: "user prompt"

      ## Correct examples:
      - For retriever_vector_db: Action Input: "user prompt"

      ## Observation: [Result of the tool]
      ... [Repeat process if necessary]
      Final Answer: The best answer you found.

      ## For each query:
      1. Analyze which information do you need.
      2. Use the available tools.
      3. Return the answer that best answers the query.

      ## IMPORTANT:
      - The action input is **always** the user prompt
      - Never modify the user prompt
      - Always follow the exact format of the examples for each tool.

      # Example Interaction:
      ## Query 1:
      "How many fans does the game have?"
      Thought: I need to find information about the number of fans the game has.
      Action: retriever_dataframe_db
      Action Input: "How many fans does the game have?"
      Observation: "1828"
      Final Answer: "The game has 1828 fans."

      ## Query 2:
      "What should be done at the beginning of the game?"
      Thought: I need to find information about the mechanics of the game, which doesn’t involve people or statistics. Therefore, I will use the vector database and look for the most relevant category (description, glossary, mechanics, objective, general rules, solo player rules), in this case, mechanics.
      Action: retriever_vector_db
      Action Input: "What should be done at the beginning of the game?"
      Observation: "At the start of the game, players..."
      Final Answer: "At the start of the game, players..."

      ## Query 3:
      "Who worked on the game?"
      Thought: I need to find information about who worked on the game. This involves people, so I will use the graph database.
      Action: retriever_graph_db
      Action Input: "Who worked on the game?"
      Observation: ['Javy M.', 'Cris K.', 'Maury M.']
      Final Answer: The team behind the game consists of Javy M., Cris K., and Maury M.
      """,
      react_chat_history=False,
      context="""
      You are a helpful assistant who always responds with truthful, helpful and fact-based answers.
      You are an expert in board games and understand their descriptive characteristics.
      Remember the outputs of the tools to use them in your final answer.
      """)

  try:
      if not user_input.strip():
          return "The query is empty."
      response = agent.chat(user_input)
  except Exception as e:
      response = f"Error processing the query: {str(e)}"


  print(f"\nAnswer: {response}")
  print("\n\n")

In [150]:
client = InferenceClient(api_key=userdata.get('HF_TOKEN'))
agent_help()

Rajas of the Ganges - AGENT HELP!
Ask me something: How many players do I need to start a game?


> Running step 93000717-5ce3-4bf8-ad6c-9c8d8126a943. Step input: How many players do I need to start a game?

Answer: Error processing the query: timed out





In [147]:
import requests
response = requests.get('https://api-inference.huggingface.co/models/gpt2')
print(response.status_code)

200
